<a href="https://colab.research.google.com/github/Anaphase21/LoRA-fine-tuning/blob/main/LoRA_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wisdom Aduah**



#Imports

In [1]:
!pip install -q loralib
!pip install datasets --upgrade
!pip install livelossplot
!pip install colorama  # print colors :).
!pip install transformers -U
!pip install bert_score
!pip install evaluate
!pip install sacrebleu

!pip install datasets
!pip install peft
!pip accelerate

import os
import math
import urllib.request

# https://stackoverflow.com/questions/68340858/in-google-colab-is-there-a-programing-way-to-check-which-runtime-like-gpu-or-tpu
# if os.environ["COLAB_GPU"] and int(os.environ["COLAB_GPU"]) > 0:
#     print("a GPU is connected.")
# else:
#     print("Only CPU accelerator is connected.")

# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html#gpu-memory-allocation
# Avoid GPU memory allocation to be done by JAX.
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

# note that jax might not run on a GPU here in colab, however we are not actually training jax models in this notebook
import chex
import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax

import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel
import datasets
import peft

from PIL import Image
from livelossplot import PlotLosses

# Utils.
import colorama

import torch
#import torchvision

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import itertools
import random

import copy

import gensim
from nltk.data import find
import nltk

nltk.download("word2vec_sample")

import huggingface_hub
import ipywidgets as widgets
from IPython.display import display

from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

from bert_score import score
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import notebook_login
from IPython.display import display, Markdown

import evaluate


!pip install livelossplot
!pip install colorama  # print colors :).
! pip install sacrebleu
! pip install evaluate

#!pip install transformers datasets
#!pip install seaborn umap-learn
#!pip install livelossplot
!pip install datasets
#!pip install -q transformers[torch]
#!pip install accelerate -U
!pip install peft

# Python utils
#!pip install -q ipdb      # debugging.
#!pip install -q colorama  # print colors :).

import os
import math
import urllib.request

import chex
import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax

import transformers
from transformers import pipeline, AutoTokenizer, AutoModel
import datasets
import peft

from PIL import Image
from livelossplot import PlotLosses
import evaluate

# Utils.
import colorama

import torch
#import torchvision

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import itertools
import random

import copy

import gensim
from nltk.data import find
import nltk


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup,get_cosine_with_hard_restarts_schedule_with_warmup,get_constant_schedule
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import re
import string
import unicodedata
import re

import huggingface_hub
import ipywidgets as widgets
from IPython.display import display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 117.7 MB/s eta 0:00:00


[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


# Function definitions

#### Function to load the BLOOM pretrained model from Hugging.

In [2]:
def load_pretrained_model(device: str):
    model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b",
                                             torch_dtype = torch.float32,
                                             device_map = device)
    tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b", padding_side = "left")
    tokenizer.pad_token = tokenizer.eos_token
    # Return the pretrained model and tokenizer for the model.
    return model, tokenizer

#### This utility function prints all trainable parameters of a given model.

In [3]:
def print_trainable_parameters(model):
  trainable_params = 0
  all_params = 0
  for _, param in model.named_parameters():
    all_params += param.numel()
    if(param.requires_grad):
      trainable_params += param.numel()
  print(f"Trainable params: {trainable_params} || All params: {all_params} || Trainable %: {100 * trainable_params / all_params}")

Here we define functions to create a prompt string, and another function to map the input and targets to a prompt string. We want to format our prompts in the form ###INPUT: {input} TARGET {target} for the model, to make it easy for the model to learn where the input is, and where the target is.

In [4]:
def create_prompt(inputs, targets):
    prompt_template = f"""###INPUT: {inputs}
###TARGET: {targets}</s>"""

    return prompt_template

def map_dataset_to_prompt(dataset, max_length):
    ds = dataset.map(lambda example: {'inputs': example['inputs'], 'targets': example['targets']}, remove_columns=list(dataset.features.keys()))
    mapped_dataset = ds.map(lambda samples: tokenizer(create_prompt(samples["inputs"], samples["targets"]), max_length = max_length, truncation = True))
    return mapped_dataset

Here, we define the hyperparameters for training.

In [5]:
batch_size = 1
gradient_accumulation_steps = 4
warmup_steps = 50
max_steps = 50
LR = 1e-3
logging_steps = 1
output_dir = "output"

In [6]:
def train_model(tokenized_dataset):
    trainer = transformers.Trainer(model = model,
                                    train_dataset = tokenized_dataset,
                                    args = transformers.TrainingArguments(per_device_train_batch_size = batch_size,
                                                                          gradient_accumulation_steps = gradient_accumulation_steps,
                                                                          warmup_steps = warmup_steps,
                                                                          max_steps = max_steps,
                                                                          learning_rate = LR,
                                                                          fp16 = False,
                                                                          logging_steps = logging_steps,
                                                                          num_train_epochs = 10,
                                                                          output_dir = output_dir),
                                    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
                                    )
    model.config.use_cache = False #Silence warnings
    trainer.train()
    return model

Function to decode generated text from the model.

In [7]:
def make_inference(inputs):
    template = f"""###INPUT: {inputs}
###TARGET: """
    batch = tokenizer(template, return_tensors = "pt", return_token_type_ids = False, padding=True).to("cuda")#, return_token_type_ids = False, padding = True).to("cuda")
    with torch.amp.autocast('cuda'):
        output_tokens = model.generate(**batch, max_new_tokens = 256, num_beams = 1, do_sample = True, temperature = 0.9)#, top_k = 2048, top_p = 0.1)
    prediction = tokenizer.decode(output_tokens[0], skip_special_tokens = True)
    return prediction

Compute the chrF++ score.

In [8]:
def compute_chrF_plus_plus(predictions, reference):
    chrf = evaluate.load("chrf")
    results = chrf.compute(predictions=predictions, references = reference)
    return results

def extract_prediction(model_output):
    prediction = ""
    segments = model_output.split("###TARGET: ")
    if len(segments) > 1:
        prediction = segments[1].strip()
    return prediction

In [9]:
def Bertscore(output,reference,tokenizer, model):

    inputs1 = tokenizer(output, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(reference, return_tensors="pt", padding=True, truncation=True)

    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

    #print("Similarity between the texts: {:.4f}".format(similarity[0][0]))

    return similarity[0][0]

In [10]:
def generate(eval_dataset):
    i = 0
    chrF_score = []
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    bert_model = BertModel.from_pretrained("bert-base-uncased")
    bert_scores = []

    for zd in eval_dataset:
        prediction_list = []
        reference_list = []
        target = zd['inputs']
        prediction = make_inference(target)
        bert_scores.append(Bertscore(prediction, target, bert_tokenizer, bert_model))
        #print(Bertscore_v2(prediction,target))
        prediction_list.append(extract_prediction(prediction))
        temp = []
        temp.append(target)
        reference_list.append(temp)
        chrF_score.append(compute_chrF_plus_plus(prediction_list, reference_list).get("score"))
        if i < 8:
            prediction = prediction.replace("###INPUT:", "### INPUT:\n").replace("###TARGET:", "### RESPONSE:\n")
            display(Markdown(prediction))
        else:
            break
        i += 1
        # if i == 8:
        #    break
    return sum(chrF_score) / len(chrF_score), sum(bert_scores) / len(bert_scores)

Load our pretrained model and tokenizer.

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model, tokenizer = load_pretrained_model(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print_trainable_parameters(model)

Trainable params: 3002557440 || All params: 3002557440 || Trainable %: 100.0


# CHRF++ score

In [ ]:
def chrfpp_score(output,reference):
  output=[output]
  reference=[[reference]]
  chrf = evaluate.load("evaluate-metric/chrf")
  result = chrf.compute(predictions=output,
                         references=reference,
                         word_order=2,
                         lowercase=True)
  return result


# LoRA

## Load our datasets.

In [ ]:
from datasets import concatenate_datasets
# Load the annotations dataset

aya_dataset = load_dataset("CohereForAI/aya_dataset", split='train')
def filter_by_language(example):
    return example['language'] == "English"

# Filter rows based on language
train_dataset = aya_dataset.filter(filter_by_language)
train_dataset = map_dataset_to_prompt(train_dataset, 256)
#print(tokenizer.decode(train_dataset[0]['input_ids']))
train_dataset = train_dataset.remove_columns(list(['inputs', 'targets']))

In [ ]:
def filter_by_language2(example):
    return example['language'] == "Zulu"

zul_dataset = aya_dataset.filter(filter_by_language2)
zul_dataset = map_dataset_to_prompt(zul_dataset, 1024)
zul_dataset = zul_dataset.remove_columns(list(['inputs', 'targets']))
final_dataset = concatenate_datasets([train_dataset, zul_dataset])

print(zul_dataset)
print(train_dataset)
print(final_dataset)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1833
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3944
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5777
})


We print one of the training examples in the Zulu language to have an idea about the length of strings we are dealing with. As you can see from the output, the input and target strings are very long, hence we chose the max_length argument to the tokenizer to be 1024.

In [ ]:
print(tokenizer.decode(train_dataset[20]['input_ids']))

###INPUT: What is light reflection?
###TARGET: Light reflection is the process by which light waves bounce off a surface. When light encounters a reflective surface, such as a mirror or smooth water, it changes direction and returns into the medium from which it originated, instead of being absorbed by the surface. This phenomenon follows the law of reflection, which states that the angle of incidence (the angle at which the light hits the surface) is equal to the angle of reflection (the angle at which the light is reflected). Reflection is responsible for many everyday visual experiences, including seeing objects in a mirror or the shimmering of light on water.</s>


## Prompt the pretrained model.

Here we try to prompt the pretrained model to observe the output before fine tuning. We expect to improve the output quality after fine tuning.

In [ ]:
print("What is matter?")
batch = tokenizer("What is matter?", return_tensors = "pt", return_token_type_ids = False).to("cuda")#, padding = True).to("cuda")
with torch.amp.autocast('cuda'):
    output_tokens = model.generate(**batch, max_new_tokens = 256, num_beams = 1, do_sample = True, temperature = 2.0)#, top_k = 2048, top_p = 0.1)
prediction = tokenizer.decode(output_tokens[0], skip_special_tokens = True)
print(f"\n{prediction}")

What is matter?

What is matter? One important type that matter refers to something that happens at all – but which only involves us is thought about as reality as only there are concrete realities. By talking about things you, as individuals have, or thoughtful reflection on some point you, for example, in fact. The latter requires that all beings were a thing so for me being like myself does give out the thought as being like I (meaning we cannot and therefore never feel). From my stand as man to be an earthling I would never have been like as it was (although what was like and for me might not be different). A man with eyes from a distance will always have some similar point in him which you do think might be interesting for you as well to know than in order to be you are someone completely separate who is, also, as everyone you see to have things in common without feeling a bit ony why or ony with this in mind being a real Earthbound.
Even your actions as a member of that I are or 

In [ ]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

## Configuration of LoRA and training.

In [ ]:
config = LoraConfig(r = 64,
                    lora_alpha = 16,
                    target_modules = ['query_key_value'],
                    lora_dropout = 0.1,
                    bias = "lora_only",
                    task_type = "CAUSAL_LM")
model = get_peft_model(model, config)
print_trainable_parameters(model)
model = train_model(train_dataset)

max_steps is given, it will override any value given in num_train_epochs


Trainable params: 19891200 || All params: 3022218240 || Trainable %: 0.6581655731122845


Step,Training Loss
1,2.711600
2,3.180500
3,3.609000
4,3.167500
5,2.611300
6,2.009800
7,4.114900
8,1.980500
9,2.715800
10,2.499400


In [ ]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
 

In [ ]:
print_trainable_parameters(model)

Trainable params: 9437184 || All params: 2859194368 || Trainable %: 0.33006444422319176


## Save our model locally in Colab.

In [ ]:
model_name = "bloom_for_zul"

In [ ]:
model.save_pretrained(model_name, safe_serialization = False)

In [ ]:
train_dataset

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 3944
})

## Load our saved model

In [ ]:
pre_model, tokenizer = load_pretrained_model(device)
model = PeftModel.from_pretrained(pre_model, model_name)
model = model.to(device)

## Load evaluation dataset

In [ ]:
def filter_by_language_code(example):
    return example['language'] == "eng"
# Filter rows based on language
eval_dataset = load_dataset("CohereForAI/aya_evaluation_suite", "dolly_machine_translated", split = "test")
eval_dataset = eval_dataset.filter(filter_by_language_code)

In [ ]:
print(eval_dataset)

Dataset({
    features: ['id', 'inputs', 'targets', 'language', 'script', 'source_id'],
    num_rows: 200
})


## Finally Generate.
Here, we call generate() to generate text for sample inputs. The function also computes the average chrF++ score for all 200 test cases.

In [ ]:
generate(eval_dataset)

### INPUT:
 What are 5 ways to eat apples?
### RESPONSE:
  1. Eat them raw, with their skin. 2. Eat them raw, with their skin and seeds. 3. Eat them raw, with their skin and seeds. 4. Eat them raw, with their skin and seeds. 5. Eat them raw, with their skin and seeds.
### RESPONSE:
  1. Eat them raw, with their skin. 2. Eat them raw, with their skin and seeds. 3. Eat them raw, with their skin and seeds. 4. Eat them raw, with their skin and seeds. 5. Eat them raw, with their skin and seeds.
### RESPONSE:
  1. Eat them raw, with their skin. 2. Eat them raw, with their skin and seeds. 3. Eat them raw, with their skin and seeds. 4. Eat them raw, with their skin and seeds. 5. Eat them raw, with their skin and seeds.
### RESPONSE:
  1. Eat them raw, with their skin. 2. Eat them raw, with their skin and seeds. 3. Eat them raw, with their skin and seeds. 4. Eat them raw, with their skin and seeds. 5. Eat them

### INPUT:
 How to escape from a helicopter trapped in water ?
### RESPONSE:
  You need to climb on the top of the helicopter and release the parachute. Then, you can float down on the water surface and escape from the helicopter. You can also jump into the water to escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also use the parachute to float down and escape from the helicopter. You can also

### INPUT:
 Provide a list of ways musicians can earn income from their music craft.
### RESPONSE:
  Musician can earn income from their music craft through various ways, such as:

1. Performing as a solo artist
2. Performing as a part of a band
3. Performing as a songwriter
4. Performing as a recording artist
5. Performing as a producer
6. Performing as a touring artist
7. Performing as a DJ
8. Performing as a music teacher
9. Performing as a record label
10. Performing as a music festival promoter
11. Performing as a music publisher
12. Performing as a music industry consultant
13. Performing as a music industry analyst
14. Performing as a music industry manager
15. Performing as a music industry consultant
16. Performing as a music industry analyst
17. Performing as a music industry manager
18. Performing as a music industry consultant
19. Performing as a music industry analyst
20. Performing as a music industry manager
21. Performing as a music industry consultant
22. Performing as a music industry analyst
23. Performing as a music industry manager
24. Performing as a music industry consultant
25. Performing as a music industry analyst

### INPUT:
 How can you open a can without using your hands?
### RESPONSE:
  Open the can with a screwdriver or a toothpick. The screwdriver is easier to use than the toothpick, but the screwdriver can break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can if it is too hard to use. The screwdriver is also more likely to damage the can than the toothpick. The screwdriver can also break the can